# This notebook contains an example of running CiSSA on auto mode.
## The only parameter that the user MUST set is the window length, L.
Even if the user does not set L explicitly, auto-cissa will take L to be half the time series length.  
If needed, auto-cissa will:
- a) Fix any censored data
- b) Replace any NaN values using cissa gap fill method
- c) Fit the data using CiSSA
- d) Find significant components using a Monte-Carlo procedure
- e) Group components into trend, periodic, and noise groups.
- f) Run frequency time analysis, plot trend and other figures.

In [1]:
#importing required packages
import pandas as pd
import numpy as np
import os
os.chdir('../../..')
home_path = os.path.abspath('')
from pycissa import Cissa
os.chdir(home_path)

In [2]:
## Read data file  (source is the Ministry of Economy of Spain.)
data = pd.read_excel(home_path + r'/data/EPC.xlsx',sheet_name = 'Data')

In [3]:
#create the cissa object
cissa_object = Cissa(t=list(range(0,len(data['Date']))),x=data['Electric power consumption in Spain'])

In [ ]:
#here we choose a window length of 5 years.
cissa_object.auto_cissa(L = 12*5,
                       K_surrogates = 5, alpha = 0.05,  #settings some parameters for the Monte-Carlo routine
                       )

Checking for censored or nan data...
RUNNING CISSA!
Performing monte-carlo significance analysis...


### Firstly, let us view the original time series and how the CiSSA method has split the components:

In [ ]:
cissa_object.figures.get('cissa').get('figure_split_components')

In [ ]:
print('''COMPONENT VARIANCE''')
print(f'''TREND   : {cissa_object.results.get('cissa').get('noise component tests').get('trend_share')}%''')
print(f'''PERIODIC: {cissa_object.results.get('cissa').get('noise component tests').get('periodic_share')}%''')
print(f'''NOISE   : {cissa_object.results.get('cissa').get('noise component tests').get('noise_share')}%''')

### Now we will look at the significant components:

In [ ]:
cissa_object.figures.get('cissa').get('figure_monte_carlo')

In [ ]:
#here we iterate through the results and find the frequencies that passed monte-carlo test.
#Note the result dictionary is structured according to input options.
#We also convert the frequency to period in years since we have monthly data
print("Trend is included in significant components.")
print(" ")
for key_j in cissa_object.results.get('cissa').get('components').keys():
    if key_j != 'trend':
        if cissa_object.results.get('cissa').get('components').get(key_j).get('monte_carlo').get('random_permutation').get('alpha').get(0.05).get('pass'):
            print("Significant CiSSA component found!")
            print(f" Frequency = {key_j} per unit timestep")
            print(f"Period = {1/(12*key_j)} years")
            print(" ")

### Next we will look at the trend...

Since L is set at 5 years (L=12*5) then trend is defined as any signal component with period longer than 5 years

In [ ]:
cissa_object.figures.get('cissa').get('figure_trend')

### Next we will look at the frequency time analysis...

In [ ]:
cissa_object.figures.get('cissa').get('figure_frequency_time')

The 1 year signal seems to have the strongest signal power, but the six month signal power greatly increases towards the end of the time series.  

### Looking now at the noise residuals

In [ ]:
cissa_object.figures.get('cissa').get('figure_residual_check')

### We can access the last 5 values of the trend, periodic, and noise components

In [ ]:
print("TREND")
print(np.column_stack((cissa_object.t[-5:].astype(str), cissa_object.x_trend[-5:].astype(str))))
print(" ")

print("PERIODIC")
print(np.column_stack((cissa_object.t[-5:].astype(str), cissa_object.x_periodic[-5:].astype(str))))
print(" ")

print("NOISE/RESIDUAL")
print(np.column_stack((cissa_object.t[-5:].astype(str), cissa_object.x_noise[-5:].astype(str))))
print(" ")


These components can be used to detrend, denoise a signal, or used in a prediction model.